# Perceptron

In [1]:
import torch
from torch import nn

x = torch.tensor([1.])
print(x)
model = nn.Linear(1,1)
#print(model.weight)
#print(model.bias)
print(model(x))
y = x @ model.weight + model.bias # perceptron why? entropy
print(y)

c:\Users\yuncs\anaconda3\envs\yolov7\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


tensor([1.])
tensor([-0.5659], grad_fn=<AddBackward0>)
tensor([-0.5659], grad_fn=<AddBackward0>)


# Multi-Layer Perceptron, MLP

In [3]:
fc1 = nn.Linear(1,100)
fc2 = nn.Linear(100,1)

x = torch.tensor([1.])
print(fc2(fc1(x)))
y = (x @ fc1.weight.T + fc1.bias) @ fc2.weight.T + fc2.bias
print(y)

model = nn.Sequential(nn.Linear(1,100), 
                      nn.Linear(100,1))
print(model(x))

# create model class
class my_model(nn.Module):
    def __init__(self, *args, **kwargs) -> None:
        super().__init__(*args, **kwargs)

        self.my_layer = nn.Sequential(nn.Linear(1,100), 
                                      nn.Linear(100,1))
    
    def forward(self, x):
        x = self.my_layer(x)
        return x

model = my_model()
x = torch.tensor([1.])
y = model(x)
print(y)

tensor([-0.1097], grad_fn=<AddBackward0>)
tensor([-0.1097], grad_fn=<AddBackward0>)
tensor([0.2639], grad_fn=<AddBackward0>)
tensor([0.1194], grad_fn=<AddBackward0>)


# MLP train using dataset 

In [1]:
import pandas as pd
import numpy as np
import torch
from torch import nn
from torch import optim
import torch.nn.functional as func
from sklearn.model_selection import train_test_split
#from sklearn.linear_model import LinearRegression # using scikit-learn

# 01 loading dataset and convert tensor
csv_file = pd.read_csv('data/Real_estate.csv')
csv_file.drop('No', inplace=True, axis=1)
y = csv_file['Y house price of unit area']
x = csv_file.drop('Y house price of unit area', axis=1) 
#print(x.shape, y.shape)
X_train, X_test, Y_train, Y_test = train_test_split(x, y, test_size=0.2, random_state=3, shuffle=True)
#print(X_train.shape, X_test.shape, Y_train.shape, Y_test.shape)
X_train = torch.tensor(X_train.values, dtype=torch.float32)
X_test = torch.tensor(X_test.values, dtype=torch.float32)
Y_train = torch.tensor(Y_train.values, dtype=torch.float32).unsqueeze(1)
Y_test = torch.tensor(Y_test.values, dtype=torch.float32).unsqueeze(1)
print(X_train.shape, X_test.shape, Y_train.shape, Y_test.shape)

""" # using scikit-learn
model = LinearRegression()
model.fit(X_train, Y_train)
y_predict = model.predict(X_test)
print(pd.DataFrame({'Y_test':Y_test, 'Y_predict':y_predict}).head())
"""

# 02 create model class
class my_model(nn.Module):
    def __init__(self):
        super().__init__()

        self.my_layer = nn.Sequential(nn.Linear(6, 30),
                                      nn.Sigmoid(),
                                      nn.Linear(30, 100),
                                      nn.Sigmoid(),
                                      nn.Sequential(*[i for _ in range(100) for i in [nn.Linear(100,100), nn.Sigmoid()]]),
                                      nn.Linear(100,1),
                                      nn.Sigmoid())
    
    def forward(self, x):
        x = self.my_layer(x)
        return x

model = my_model()

# 03 train created model
epoch = 50 # how many train count?
LR = 1e-7 # learning rate -> how many update weight?
optimizer = optim.SGD(model.parameters(), lr=LR, momentum=0.9)

model.train() # model train mode
for ep in range(epoch):
    y_h = model(X_train) # forward network
    loss_func = func.binary_cross_entropy(y_h, Y_train) # loss function
    loss_func_tmp = -(torch.sum(torch.log(y_h**Y_train * (1-y_h)**(1-Y_train)))/331) # loss function
    optimizer.zero_grad() # prevent gradient accumulate
    loss_func.backward() # backward network (backpropagation)
    optimizer.step() # weight update (backpropagation)
    # train loss print
    print(f"epoch: {ep}, train_loss: {round(loss_func.item(), 3)}, my_loss: {round(loss_func_tmp.item(), 3)}")



c:\Users\yuncs\anaconda3\envs\yolov7\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


torch.Size([331, 6]) torch.Size([83, 6]) torch.Size([331, 1]) torch.Size([83, 1])
epoch: 0, train_loss: 11.553, my_loss: 11.552
epoch: 1, train_loss: 11.549, my_loss: 11.549
epoch: 2, train_loss: 11.541, my_loss: 11.541
epoch: 3, train_loss: 11.531, my_loss: 11.531
epoch: 4, train_loss: 11.518, my_loss: 11.518
epoch: 5, train_loss: 11.502, my_loss: 11.502
epoch: 6, train_loss: 11.485, my_loss: 11.485
epoch: 7, train_loss: 11.465, my_loss: 11.465
epoch: 8, train_loss: 11.443, my_loss: 11.443
epoch: 9, train_loss: 11.42, my_loss: 11.42
epoch: 10, train_loss: 11.395, my_loss: 11.395
epoch: 11, train_loss: 11.369, my_loss: 11.369
epoch: 12, train_loss: 11.341, my_loss: 11.341
epoch: 13, train_loss: 11.313, my_loss: 11.313
epoch: 14, train_loss: 11.284, my_loss: 11.284
epoch: 15, train_loss: 11.253, my_loss: 11.253
epoch: 16, train_loss: 11.222, my_loss: 11.222
epoch: 17, train_loss: 11.191, my_loss: 11.191
epoch: 18, train_loss: 11.158, my_loss: 11.158
epoch: 19, train_loss: 11.125, my_los

In [46]:
# prevent gradient accumulate
x = torch.tensor([1.], requires_grad=True)
for _ in range(3):
    loss = x**2 # 식
    loss.backward() # 미분 +=
    print(x.grad) # 미분결과
    x.grad = None

tensor([2.])
tensor([2.])
tensor([2.])
